# Limit & Market Order Book

In [1]:
class Order:
    def __init__(self, order_id, side, price=None, quantity=0, order_type="limit"):
        self.order_id = order_id
        self.side = side  # 'buy' or 'sell'
        self.price = price  # None for market orders
        self.quantity = quantity
        self.order_type = order_type  # 'limit' or 'market'

class LimitOrderBook:
    def __init__(self):
        self.buy_orders = []  
        self.sell_orders = []

    def place_order(self, order_id, side, price=None, quantity=0, order_type="limit"):
        order = Order(order_id, side, price, quantity, order_type)

        if order_type == "limit":
            if side == "buy":
                self.buy_orders.append(order)
                self.buy_orders.sort(key=lambda o: -o.price)  # Sort by descending price
            else:
                self.sell_orders.append(order)
                self.sell_orders.sort(key=lambda o: o.price)  # Sort by ascending price
        elif order_type == "market":
            self.match_market_order(order)
        else:
            return "Invalid order type."

        self.match_orders()  # Match any possible limit orders after placing
        return f"Order {order_id} placed."

    def cancel_order(self, order_id):
        for order_list in [self.buy_orders, self.sell_orders]:
            for order in order_list:
                if order.order_id == order_id:
                    order_list.remove(order)
                    return f"Order {order_id} canceled."
        return f"Order {order_id} not found."

    def match_market_order(self, market_order):
        if market_order.side == "buy":
            while market_order.quantity > 0 and self.sell_orders:
                top_sell = self.sell_orders[0]
                if market_order.quantity <= top_sell.quantity:
                    print(f"Market Buy: {market_order.quantity} units at ${top_sell.price:.2f}")
                    top_sell.quantity -= market_order.quantity
                    market_order.quantity = 0
                    if top_sell.quantity == 0:
                        self.sell_orders.pop(0)
                else:
                    print(f"Market Buy: {top_sell.quantity} units at ${top_sell.price:.2f}")
                    market_order.quantity -= top_sell.quantity
                    self.sell_orders.pop(0)
        elif market_order.side == "sell":
            while market_order.quantity > 0 and self.buy_orders:
                top_buy = self.buy_orders[0]
                if market_order.quantity <= top_buy.quantity:
                    print(f"Market Sell: {market_order.quantity} units at ${top_buy.price:.2f}")
                    top_buy.quantity -= market_order.quantity
                    market_order.quantity = 0
                    if top_buy.quantity == 0:
                        self.buy_orders.pop(0)
                else:
                    print(f"Market Sell: {top_buy.quantity} units at ${top_buy.price:.2f}")
                    market_order.quantity -= top_buy.quantity
                    self.buy_orders.pop(0)
        else:
            print("No matching orders for market order.")

    def match_orders(self):
        while self.buy_orders and self.sell_orders:
            top_buy = self.buy_orders[0]
            top_sell = self.sell_orders[0]

            if top_buy.price >= top_sell.price:  # Match if buy price >= sell price
                match_quantity = min(top_buy.quantity, top_sell.quantity)
                print(f"Matched: {match_quantity} units at ${top_sell.price:.2f}")

                top_buy.quantity -= match_quantity
                top_sell.quantity -= match_quantity

                # Remove fully filled orders
                if top_buy.quantity == 0:
                    self.buy_orders.pop(0)
                if top_sell.quantity == 0:
                    self.sell_orders.pop(0)
            else:
                break

    def display_order_book(self):
        print("\nOrder Book:")
        print("Buy Orders:")
        for order in self.buy_orders:
            print(f"{order.quantity} @ ${order.price:.2f}")
        print("\nSell Orders:")
        for order in self.sell_orders:
            print(f"{order.quantity} @ ${order.price:.2f}")


# Example
if __name__ == "__main__":
    book = LimitOrderBook()

    book.place_order("1", "buy", 100, 10)
    book.place_order("2", "buy", 105, 5)
    book.place_order("3", "sell", 103, 7)
    book.place_order("4", "sell", 110, 8)

    print("\n--- Initial Order Book ---")
    book.display_order_book()

    # Placing a market buy order
    print("\n--- Market Buy Order ---")
    book.place_order("5", "buy", quantity=6, order_type="market")
    book.display_order_book()

    # Placing a market sell order
    print("\n--- Market Sell Order ---")
    book.place_order("6", "sell", quantity=5, order_type="market")
    book.display_order_book()


Matched: 5 units at $103.00

--- Initial Order Book ---

Order Book:
Buy Orders:
10 @ $100.00

Sell Orders:
2 @ $103.00
8 @ $110.00

--- Market Buy Order ---
Market Buy: 2 units at $103.00
Market Buy: 4 units at $110.00

Order Book:
Buy Orders:
10 @ $100.00

Sell Orders:
4 @ $110.00

--- Market Sell Order ---
Market Sell: 5 units at $100.00

Order Book:
Buy Orders:
5 @ $100.00

Sell Orders:
4 @ $110.00
